In [1]:
# script_hang indefinitely at points_to_convex_hull_volume_mask

import numpy as np
import stackview
from vedo import Points, ConvexHull, Volume


def points_to_convex_hull_volume_mask(points, volume_shape_zyx, dilation_radius=3) -> Volume:
    """
    Converts a set of 3D points to a binary volume mask of the inner part of the embryo using a convex hull.

    This function takes a set of 3D points and a volume shape, constructs a convex hull from the points,
    binarizes the convex hull into a volume mask, and then erodes/dilates the mask. 

    Args:
        points (numpy.ndarray): A numpy array of shape (N, 3) representing the 3D points in ZYX order.
        volume_shape_zyx (tuple): A tuple (z, y, x) representing the shape of the volume.
        dilation_radius (int): The radius of the dilation applied to the volume mask.  This expands the mask
            outwards, useful for ensuring complete coverage of the structure represented by the points.

    Returns:
        vedo.Volume: A vedo.Volume object representing the binary volume mask.  The mask has values of 255 inside
            the convex hull and 0 outside.
    """
    points_raw = points[:, [2, 1, 0]]
    pts = Points(points_raw)
    print("Creating convex hull from points")
    hull = ConvexHull(pts)

    vol_shape_xyz = volume_shape_zyx[::-1]
    print("Binarizing convex hull into volume mask")
    vol_mask = hull.binarize(values=(255,0),dims=vol_shape_xyz,spacing=[1,1,1], origin=(0,0,0))
    if dilation_radius > 0:
        print(f"Dilating with radius of {dilation_radius}")
        modified = vol_mask.clone().dilate(neighbours=(dilation_radius,dilation_radius,dilation_radius))
    else:
        erosion_radius = abs(dilation_radius)
        print(f"Eroding with erosion radius of {erosion_radius}")
        modified = vol_mask.clone().erode(neighbours=(erosion_radius,erosion_radius,erosion_radius))
    return modified

down_cropped = np.load("/scratch/artemiy/test_data_for_serosa_peeling/peeling_debug/script_hang_at__points_to_convex_hull_volume_mask/down_cropped_tp_767.npy")
print(down_cropped.shape)
points = np.load("/scratch/artemiy/test_data_for_serosa_peeling/peeling_debug/script_hang_at__points_to_convex_hull_volume_mask/tp_767_surface_points.npy") 
print(points[0,:,:])
# mod = points_to_convex_hull_volume_mask(points, volume_shape_zyx=down_cropped.shape, dilation_radius=3)

(146, 298, 621)
[ 1 22 46]


In [5]:
stackview.slice(down_cropped, continuous_update=True)

In [6]:
from wbns import substract_background

only_structures = substract_background(down_cropped, 4, 1)

In [7]:
stackview.slice(only_structures, continuous_update=True)

In [9]:
# Otsu is clearly failing hard, and whole image is 1 value

from skimage import filters
from scipy import ndimage as cpu_ndimage


substracted_bkg = only_structures
th = filters.threshold_otsu(substracted_bkg)
mask = substracted_bkg >= th

structuring_element = np.ones((3,3,3))
eroded_mask = cpu_ndimage.binary_erosion(mask, structure=structuring_element).astype(mask.dtype)  # Keep original datatype
# Zerroing out the border to remove artifacts that wbns generates
zero_y = int(eroded_mask.shape[1] * (1.15 - 1) / 2) 
zero_x = int(eroded_mask.shape[2] * (1.15 - 1) / 2)
eroded_mask[:,-zero_y:,:] = False
eroded_mask[:,:zero_y,:] = False
eroded_mask[:,:,-zero_x:] = False
eroded_mask[:,:,:zero_x] = False
stackview.slice(mask, continuous_update=True)

In [16]:
substracted_bkg = only_structures
th = filters.threshold_mean(substracted_bkg)
mask = substracted_bkg >= th
stackview.slice(mask, continuous_update=True)

In [2]:
points.shape

(23110952, 3)

In [4]:
points_766 = np.load("/scratch/artemiy/test_data_for_serosa_peeling/peeling_debug/script_hang_at__points_to_convex_hull_volume_mask/tp_766_surface_points.npy")
points_766.shape

(92905, 3)